# RAG w/ Gemini

#### *RAG 프로세스: 문서 로드 및 Vector DB 저장*
1. 문서 로드 (load): 문서(pdf, word), RAW DATA, 웹페이지, notion 등의 데이터 읽기
2. 분할 (split): 불러온 문서를 chunk 단위로 분할
3. 임베딩 (embedding): 문서를 벡터 표현으로 변환
4. 벡터DB(VectorStore): 변환된 벡터를 DB에 저장

<br>

#### *RAG 프로세스: 문서 검색 및 결과 도출*
5. 검색 (retrieval): 유사도 검색(similarity, mmr), Multi-Query, Multi-Retriever
6. 프롬프트 (prompt): 검색된 결과를 바탕으로 원하는 결과를 도출하기 위한 프롬프트
7. 모델 (LLM): 모델 선택 (gpt, gemini, etc.)
8. 결과 (output): 텍스트, JSON, markdown

<br><hr>

- 예제
  - 데이터: 한국산업은행 금융관련 용어 csv 파일
  - 임베딩: 업스테이지 모델
  - 벡터 스토어: 크로마 DB

<br>

- 변경
  - 데이터: 제주도 맛집 정보
  - 임베딩: 허깅페이스 모델
    - intfloat/multilingual-e5-large-instruct
    - jhgan/ko-sroberta-multitask
    - upskyy/bge-m3-Korean
  - 벡터스토어: 크로마 DB


In [1]:
# 랭스미스 추적
from langchain_teddynote import logging

# 프로젝트 이름 입력
logging.langsmith("bigcon_langchain_test")

# 추적을 끄고 싶은 경우
# logging.langsmith("bigcon_langchain_test", set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
bigcon_langchain_test


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# 디버깅을 위한 프로젝트명
os.environ["bigcon_langchain_test"] = "RAG TUTORIAL"

In [6]:
# 1) 문서 로드
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
  file_path="..\data\sample_1000_with_meta.csv",
  encoding="cp949"
)
pages = loader.load()
print(f"문서의 수: {len(pages)}")

# 10번째 페이지의 내용 확인
print(f"\n[페이지내용]\n{pages[10].page_content[:500]}")
print(f"\n[metadata]\n{pages[10].metadata}\n")

문서의 수: 1003

[페이지내용]
YM: 202303
MCT_NM: 주식회사웨이뷰
OP_YMD: 20230207
TYPE: T30
MCT_TYPE: 햄버거
temp_05_11: 11.25069124
temp_12_13: 13.04193548
temp_14_17: 13.04596774
temp_18_22: 11.43225806
temp_23_04: 10.67473118
TEMP_AVG: 11.88911674
latitude: 33.4073014
longitude: 126.2530211
Polygon: P5
area: 서부
ADDR: 제주 제주시 한림읍 옹포리 419-3번지
RANK_CNT: 1
RANK_AMT: 1
RANK_MEAN: 5
MON_UE_CNT_RAT: 0.1325695581
TUE_UE_CNT_RAT: 0.1309328969
WED_UE_CNT_RAT: 0.1554828151
THU_UE_CNT_RAT: 0.124386252
FRI_UE_CNT_RAT: 0.1080196399
SAT_UE_CNT_RAT:

[metadata]
{'source': '..\\data\\sample_1000_with_meta.csv', 'row': 10}



In [7]:
# 2) 허깅페이스 모델을 사용하여 임베딩 생성 
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sroberta-multitask")
# model = AutoModel.from_pretrained("jhgan/ko-sroberta-multitask")

# for page in pages:
#     text = page.page_content
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
#     embedding = model(**inputs).last_hidden_state.mean(dim=1)  # 평균 풀링으로 임베딩 생성

### >> 이 방식은 <임베딩 계산 방식을 세밀히 제어해야 하는 경우>에 추천

c:\workspaces\llmGeminiTest\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# 2) 허깅페이스 모델을 사용하여 임베딩 생성 
from langchain.embeddings import HuggingFaceEmbeddings

# HuggingFace 임베딩 클래스 사용
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

C:\Users\USER\AppData\Local\Temp\ipykernel_15508\990583999.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
c:\workspaces\llmGeminiTest\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# 3) Chroma 벡터스토어에 저장
from langchain_community.vectorstores import Chroma

# Chroma 인스턴스 생성
vectorstore = Chroma.from_documents(pages, embeddings, persist_directory="./database")

# 벡터스토어에 저장된 정보 확인
print(f"벡터스토어에 저장된 문서 수: {len(vectorstore)}")

벡터스토어에 저장된 문서 수: 2006


In [11]:
# 4) MMR 검색기 생성
# 벡터스토어 객체는 코사인 유사도/유클리디언 거리/MMR(Maximal Marginal Relevance) 등의 검색 방법이 있음
# >> MMR 방식 사용 >> 질의어와 관련성이 높으면서도 다양한 문서를 검색하기 위함

### MMR 검색 기법 작동 과정
# 1) 질의어와 관련성이 높은 문서 fetch_k 개를 가져옴
# 2) fetch_k개의 문서에 대해 이터레이션을 돌면서 질의어와 관련성이 높으면서도 이전 이터레이션에서 이미 선택된 문서와는 유사성이 낮은 문서를 가져옴
# 3) 총 k개를 가져올 때까지 2번을 반복

retriever = vectorstore.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}
)

In [42]:
# 5) 프롬프트 템플릿 생성
from langchain.prompts import ChatPromptTemplate

# 검색 결과는 {context}에, 질의는 {query}에 들어감

template = """
[context]: {context}
---
[질의]: {query}
---
[예시]
제주도 (area)에 위치한 (사용자의 요구) 맛집을 추천드립니다.
[MCT_NM]은 [area]에 위치한 [MCT_TYPE] 맛집입니다. [특징] 설명, [ADDR]에 위치하고 있습니다.
3~5개 추천
---
위의 [context] 정보 내에서 [질의]에 대해 답변 [예시]와 같이 술어를 붙여서 답하세요. 특징은 제공한 데이터 내에서만 답하세요.
"""
prompt = ChatPromptTemplate.from_template(template)

In [43]:
# 6) 제미나이 모델 객체 가져오기
from langchain_google_genai import ChatGoogleGenerativeAI

# test할 때는 답변의 일관성을 높이기 위해 일단 temp=0으로 설정
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.5)
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002BAA868CFD0>, default_metadata=())

In [44]:
# 7) 검색 결과 가공
# 원천 데이터에는 레코드, 필드 등 데이터 구조에 대한 정보 + 불필요한 메타 정보가 같이 있음
# merge_pages 함수로 원천 정보 중 page_content 정보만 가져오고 페이지 간에는 두 개의 개행을 적용하여 하나의 문서로 병합하는 작업을 수행
# 어떤 정보가 검색되었는지 확인하기 위해 print

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    print(f"참조 문서 시작==>[\n{merged}\n]<==참조 문서 끝")
    return merged

In [45]:
# 8) 체이닝 구성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트, LLM, 검색기, 검색결과 가공함수를 연결하여 체인을 구성
chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

In [46]:
# 9) 질의응답 테스트
answer = chain.invoke("우도에서 로컬 주민들에게 인기많은 맛집을 알려줘")
print(f"answer1: {answer}\n\n")

# answer = chain.invoke("제주도 북부에서 로컬 사람들에게 인기 많은 맛집을 추천해줘")
# print(f"answer2: {answer}")

# answer = chain.invoke("40대 남성들에게 인기가 많은 맛집 추천해줘")
# print(f"answer2: {answer}")

참조 문서 시작==>[
YM: 202307
MCT_NM: 제주포레스트주식회사 포메인제주노형점
OP_YMD: 20200420
TYPE: T9
MCT_TYPE: 동남아/인도음식
temp_05_11: 27.58110599
temp_12_13: 29.90322581
temp_14_17: 29.92258065
temp_18_22: 28.15612903
temp_23_04: 26.80430108
TEMP_AVG: 28.47346851
latitude: 33.4842637
longitude: 126.4798582
Polygon: P3
area: 북부
ADDR: 제주 제주시 노형동 3783-2번지 3층
RANK_CNT: 3
RANK_AMT: 3
RANK_MEAN: 4
MON_UE_CNT_RAT: 0.1475409836
TUE_UE_CNT_RAT: 0.1557377049
WED_UE_CNT_RAT: 0.1967213115
THU_UE_CNT_RAT: 0.106557377
FRI_UE_CNT_RAT: 0.1147540984
SAT_UE_CNT_RAT: 0.09836065574
SUN_UE_CNT_RAT: 0.1803278689
HR_5_11_UE_CNT_RAT: 0.04098360656
HR_12_13_UE_CNT_RAT: 0.393442623
HR_14_17_UE_CNT_RAT: 0.2704918033
HR_18_22_UE_CNT_RAT: 0.2950819672
HR_23_4_UE_CNT_RAT: 0.0
LOCAL_UE_CNT_RAT: 0.6627218935
RC_M12_MAL_CUS_CNT_RAT: 0.396
RC_M12_FME_CUS_CNT_RAT: 0.604
RC_M12_AGE_UND_20_CUS_CNT_RAT: 0.138
RC_M12_AGE_30_CUS_CNT_RAT: 0.268
RC_M12_AGE_40_CUS_CNT_RAT: 0.351
RC_M12_AGE_50_CUS_CNT_RAT: 0.18
RC_M12_AGE_OVR_60_CUS_CNT_RAT: 0.064

YM: 

In [30]:
import pandas as pd

df = pd.read_csv("..\data\sample_1000_with_meta.csv", encoding='cp949')

In [36]:
df['MCT_TYPE']

0                업종
1            STRING
2       요식관련 30개 업종
3               햄버거
4                일식
           ...     
998             가정식
999         단품요리 전문
1000            햄버거
1001             피자
1002           스테이크
Name: MCT_TYPE, Length: 1003, dtype: object

In [5]:
# 3) 벡터스토어에 저장 (ChromaDB)
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# 벡터스토어 생성
vectorstore = Chroma(embedding_function=lambda x: model(**tokenizer(x, return_tensors="pt", padding=True, truncation=True)).last_hidden_state.mean(dim=1).detach().numpy())

# 4) 벡터와 메타데이터를 벡터스토어에 저장
for page in pages:
    text = page.page_content
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    embedding = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()  # 임베딩을 NumPy 배열로 변환
    
    vectorstore.add_texts(
        texts=[text],  # 텍스트 데이터
        metadatas=[{"metadata": page.metadata}],  # 메타데이터
        embeddings=[embedding]  # 임베딩
    )

print("벡터스토어에 저장 완료")

C:\Users\USER\AppData\Local\Temp\ipykernel_25532\425915403.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=lambda x: model(**tokenizer(x, return_tensors="pt", padding=True, truncation=True)).last_hidden_state.mean(dim=1).detach().numpy())


AttributeError: 'function' object has no attribute 'embed_documents'

In [29]:
# 3) 임베딩 & 벡터스토어 생성
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# from langchain.llms import HuggingFaceHub
# from transformers import AutoTokenizer, AutoModel

vectorstore = FAISS.from_documents(documents=pages,
                                   embedding=HuggingFaceEmbeddings()
                                   )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

c:\workspaces\llmGeminiTest\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# chroma DB 적용
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=pages, embedding=HuggingFaceEmbeddings())

In [ ]:
# 단계 4: 검색(Search)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [ ]:
# 단계 5: 프롬프트 생성(Create Prompt)
from langchain import hub

# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
# 단계 6: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [ ]:
def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


In [ ]:
# 단계 7: 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "부영그룹의 출산 장려 정책에 대해 설명해주세요"
response = rag_chain.invoke(question)

In [ ]:
# 결과 출력
print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

In [20]:
from transformers import AutoModelForCausalLM


# HuggingFace Repository ID
repo_id = "jhgan/ko-sroberta-multitask"
model = AutoModelForCausalLM.from_pretrained(
  # device_map: 모델이 GPU로 이동되도록 함
  # load_in_4bit: 리소스 요구사항을 줄이기 위해 4비트 동적 양자화를 적용
  repo_id, device_map='auto', load_in_4bit=True
)

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [15]:
Chroma.from_documents(pages, model, persist_directory="./database")

AttributeError: 'BertTokenizerFast' object has no attribute 'embed_documents'

In [20]:

import pandas as pd

df = pd.read_csv("../data/llm_sample_data_with_every_area.csv", encoding='cp949')
df.head()

,Unnamed: 0,YM,MCT_NM,OP_YMD,TYPE,MCT_TYPE,temp_05_11,temp_12_13,temp_14_17,temp_18_22,...,HR_18_22_UE_CNT_RAT,HR_23_4_UE_CNT_RAT,LOCAL_UE_CNT_RAT,RC_M12_MAL_CUS_CNT_RAT,RC_M12_FME_CUS_CNT_RAT,RC_M12_AGE_UND_20_CUS_CNT_RAT,RC_M12_AGE_30_CUS_CNT_RAT,RC_M12_AGE_40_CUS_CNT_RAT,RC_M12_AGE_50_CUS_CNT_RAT,RC_M12_AGE_OVR_60_CUS_CNT_RAT
0,58148,202311,별미돈,20211008,T1,가정식,13.142857,15.771667,15.541667,13.405333,...,1.000000,0.000000,0.800000,0.676,0.324,0.101,0.251,0.377,0.198,0.072
1,53199,202310,한라산과자점,20210316,T13,베이커리,19.402765,22.798387,22.369355,19.323226,...,0.000000,0.000000,0.051303,0.270,0.730,0.519,0.276,0.119,0.066,0.019
2,44383,202308,그럼외도,20200207,T6,단품요리 전문,28.137788,30.480645,30.813710,28.823226,...,0.029126,0.000000,0.218391,0.320,0.680,0.466,0.335,0.100,0.081,0.018
3,63422,202312,숯검댕이2호점,20180904,T1,가정식,9.402857,12.151667,11.806667,9.794667,...,0.764706,0.235294,0.226131,0.518,0.482,0.333,0.205,0.221,0.179,0.062
4,39190,202307,거멍국수,20150702,T6,단품요리 전문,25.585714,26.988710,26.583871,25.608387,...,0.081081,0.000000,0.083749,0.593,0.407,0.178,0.303,0.247,0.188,0.084


In [24]:
df[df['MCT_NM'] == '(유)아웃백스테이크하우스 제주아일랜드점']['ADDR']

496    제주 제주시 오라이동 3173번지 1층
Name: ADDR, dtype: object